# Оглавление

[1. Импорт модуля и загрузка датафрейма](#Импорт-модуля-и-загрузка-датафрейма)
<br>[2. Деление на тренировочную, валидационную и тестовую выборки](#Train,-test,-valid)
<br>[3. Лемматизция текста](#Лемматизация-текста)
<br>[4. Удаление стоп-слов](#Удаление-стоп-слов)
<br>[5. Преобразование в TF-IDF вектора](#Преобразование-в-TF-IDF-вектора)
<br>[6. Обучение модели логистической регрессии](#Обучение-модели-логистической-регрессии)
<br>[7. Запись модели в файл](#Запись-модели-в-файл)
<br>[8. Загрузка модели из файла](#Загрузка-модели-из-файла)
<br>[9. Предсказание эмоции для заданного текста](#Предсказание-эмоции-для-заданного-текста)

### Импорт модуля и загрузка датафрейма

In [8]:
import pickle
from tqdm import tqdm
from emotion_text import *

In [9]:
df = open_file('data.jsonl')

                                                text  label
0  i feel awful about it too because it s my job ...      0


## Train, test, valid
Делим датасет на тренировочную, валидационную и тестовую выборки

In [10]:
X_train, y_train, X_test, X_val, y_test, y_val = tqdm(train_test_val(df.text, df.label))

100%|██████████| 6/6 [00:00<?, ?it/s]


## Лемматизация текста

In [12]:
X_train, X_test, X_val = lemmatize_text(X_train, X_test, X_val)

KeyboardInterrupt: 

## Удаление стоп-слов

In [12]:
X_train, X_test, X_val = stopwords_text(X_train, X_test, X_val, language = 'english', num_row=1)

169233               fail play doubtless still feel jealous
228397                                       feel pressured
130131    still confusing diaper cry bored cry tends lea...
94734            reached check lane feeling highly agitated
251624    used one skein salt pepper quick patter christ...
Name: text, dtype: object

## Преобразование в TF-IDF вектора

In [ ]:
X_train_tfidf, X_test_tfidf, X_val_tfidf, vectorizer = tf_idf_vec(X_train, X_test, X_val)

## Обучение модели логистической регрессии

In [16]:
for C in tqdm(np.arange (0.1, 5.0, 0.1)):
    
    model_LogisticRegression_L2 = LogisticRegression(class_weight='balanced',
                                                     multi_class='multinomial', 
                                                     max_iter = 1000, 
                                                     solver='lbfgs', 
                                                     penalty='l2', 
                                                     C=C)
    learning_model(X_train_tfidf, y_train, X_val_tfidf, y_val, X_test_tfidf, y_test, model_LogisticRegression_L2, C)

## Запись модели в файл

In [ ]:
with open('model.pkl', 'wb') as f:
    pickle.dump(model_LogisticRegression_L2, f)

## Загрузка модели из файла

In [ ]:
with open('model.pkl', 'rb') as f:
    model = pickle.load(f)

## Предсказание эмоции для заданного текста

In [ ]:
word = ''

word_tfidf = vectorizer.transform([word])
model.predict(word_tfidf)

In [ ]:
emotions_labels = {0: 'sadness',
                    1: 'joy',
                    2: 'love',
                    3: 'anger',
                    4: 'fear',
                    5: 'surprise'}